# Testing and Evaluation 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive

!git clone https://github.com/aritra1804/Cohere-LinguaLink.git

# Cohere Installation

In [ ]:
!pip install cohere

In [ ]:
!python3 scripts/cohere_align.py \
  --cohere_api_key '<api_key>' \
  -m 'embed-multilingual-v2.0' \
  -s src.txt \
  -t trg.txt \
  -o cohere \
  --retrieval 'nn' \
  --dot \
  --cuda

# Laser Installation

In [ ]:
!pip install laserembeddings


!python -m laserembeddings download-models

In [ ]:
!python3 scripts/laser_align.py \
  -s src.txt \
  -t trg.txt \
  -o cohere \
  --src_lang ha \
  --trg_lang en \
  --retrieval 'nn' \
  --dot \
  --cuda

#Evaluation

## Download FLORES Dataset

In [ ]:
# evaluation on flores devtest

!wget --trust-server-names https://tinyurl.com/flores200dataset

In [ ]:
!tar -xvzf flores200_dataset.tar.gz

## Generate alignment using laser and cohere

In [ ]:
!python3 scripts/laser_align.py \
  -s flores200_dataset/devtest/hau_Latn.devtest \
  -t flores200_dataset/devtest/eng_Latn.devtest \
  -o cohere/flores \
  --src_lang ha \
  --trg_lang en \
  --retrieval 'nn' \
  --dot \
  --cuda

In [ ]:
!python3 scripts/cohere_align.py \
  --cohere_api_key '<api_key>' \
  -m 'embed-multilingual-v2.0' \
  -s flores200_dataset/devtest/hau_Latn.devtest \
  -t flores200_dataset/devtest/eng_Latn.devtest \
  -o cohere/flores \
  --retrieval 'nn' \
  --dot \
  --cuda

## Calculate F1 score

In [ ]:
from sklearn.metrics import f1_score
import pandas as pd

with open('flores200_dataset/devtest/eng_Latn.devtest') as f:
  y_true = f.readlines()
  y_true = [y.strip() for y in y_true]

# cohere f1
df = pd.read_csv('cohere/flores/cohere_translations.csv')
y_pred = df['translations'].tolist()
cohere_f1 = f1_score(y_true, y_pred, average='macro')

# laser f1
df = pd.read_csv('cohere/flores/laser_translations.csv')
y_pred = df['translations'].tolist()
laser_f1 = f1_score(y_true, y_pred, average='macro')

# Print Results

In [ ]:
print(cohere_f1, laser_f1)